# Blogs Dataset

Running Modularity-Aware Graph Autoencoder on the Blogs dataset with specified hyperparameters.

In [ ]:
%cd modularity_aware_gae
!python train.py --dataset=blogs --features=False --task=task_2 --model=gcn_vae --iterations=200 --learning_rate=0.01 --hidden=32 --dimension=16 --beta=0.75 --lamb=0.5 --gamma=2 --s_reg=10 --fastgae=False --nb_run=1
%cd ..

c:\Users\Moher\Personal\PhD\Projects\modularity_aware_gae\modularity_aware_gae

 
 
 
[MODULARITY-AWARE GRAPH AUTOENCODERS]
 
 
 

EXPERIMENTAL SETTING 

- Graph dataset: blogs
- Mode name: gcn_vae
- Number of models to train: 1
- Number of training iterations for each model: 200
- Learning rate: 0.01
- Dropout rate: 0.0
- Use of node features in the input layer: False
- Dimension of the GCN hidden layer: 32
- Dimension of the output layer: 16
- lambda: 0.5
- beta: 0.75
- gamma: 2.0
- s: 10
- FastGAE: no 

Final embedding vectors will be evaluated on:
- Task 2, i.e., joint community detection and link prediction

 
 
 

LOADING DATA

Loading the blogs graph
- Number of nodes: 1224
- Number of communities: 2
- Use of node features: False
Done! 
 
 
 

EXPERIMENTS ON MODEL 1 / 1 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 5.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running 

2025-02-25 22:17:28.555990: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_100.dll'; dlerror: cudart64_100.dll not found
2025-02-25 22:17:28.556248: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-02-25 22:17:31.987150: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library nvcuda.dll
2025-02-25 22:17:32.008473: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 4080 major: 8 minor: 9 memoryClockRate(GHz): 2.505
pciBusID: 0000:01:00.0
2025-02-25 22:17:32.009397: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_100.dll'; dlerror: cudart64_100.dll not found
2025-02-25 22:17:32.010313: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cubl